In [195]:
import pandas as pd
import sqlite3


In [298]:
def return_table(begin_date2, end_date2):
    # Set up sqlite
    connection = sqlite3.connect('../static/data/sensors_readings_2016_present.db')
    
    # Assemble Query with proper placement of WHERE clause and using parameterized queries
    sql_query = """
    SELECT sensor_id, latitude, longitude, altitude, AVG(pm2) AS avg_pm2, AVG(pm10) AS avg_pm10
    FROM sensors_readings
    WHERE (date BETWEEN ? AND ?)
    GROUP BY sensor_id, latitude, longitude, altitude
    """
    
    # Execute the query with parameter substitution to prevent SQL injection
    df = pd.read_sql_query(sql_query, connection, params=(begin_date2, end_date2))
    
    connection.close()
    return df

In [299]:
df = return_table('2023-09-27','2023-09-30')
df

,sensor_id,latitude,longitude,altitude,avg_pm2,avg_pm10
0,77,40.750816,-111.825290,NaN,17.717333,17.952000
1,443,41.224422,-111.968376,NaN,6.603000,7.290167
2,525,40.783870,-111.870140,4891.0,0.902167,1.161167
3,984,40.595387,-111.807755,NaN,1.848500,2.114833
4,3238,40.664246,-111.849230,4331.0,3.000167,3.461167
...,...,...,...,...,...,...
195,169779,40.769947,-111.847015,4758.0,1.978167,2.797833
196,169851,40.902760,-111.872420,4361.0,2.419000,3.307500
197,169867,40.871600,-111.903200,4306.0,2.413000,3.527167
198,174555,40.722404,-111.823616,4621.0,1.970000,2.423667


In [272]:
df_color = pd.read_csv('../static/data/sensor_categories.csv')
df = pd.merge(df,df_color, on = 'sensor_id')

In [273]:
df

,sensor_id,latitude,longitude,altitude,avg_pm2,avg_pm10,category,county
0,77,40.750816,-111.825290,NaN,17.717333,17.952000,red,Salt Lake County
1,443,41.224422,-111.968376,NaN,6.603000,7.290167,red,Weber County
2,525,40.783870,-111.870140,4891.0,0.902167,1.161167,green,Salt Lake County
3,984,40.595387,-111.807755,NaN,1.848500,2.114833,blue,Salt Lake County
4,3238,40.664246,-111.849230,4331.0,3.000167,3.461167,blue,Salt Lake County
...,...,...,...,...,...,...,...,...
195,169779,40.769947,-111.847015,4758.0,1.978167,2.797833,red,Salt Lake County
196,169851,40.902760,-111.872420,4361.0,2.419000,3.307500,orange,Davis County
197,169867,40.871600,-111.903200,4306.0,2.413000,3.527167,blue,Davis County
198,174555,40.722404,-111.823616,4621.0,1.970000,2.423667,blue,Salt Lake County


In [274]:
# Compute category averages within each county
averages = df.groupby(['county', 'category']).mean().reset_index()
averages = averages[['county', 'category', 'avg_pm2', 'avg_pm10']]
averages.rename(columns={'avg_pm2': 'cat_avg_pm2', 'avg_pm10': 'cat_avg_pm10'}, inplace=True)
averages['cat_avg_pm2'] = round(averages['cat_avg_pm2']).astype('int')
averages['cat_avg_pm10'] = round(averages['cat_avg_pm10']).astype('int')

# Prepare the main DataFrame for merging
df['avg_pm2'] = round(df['avg_pm2']).astype('int')
df['avg_pm10'] = round(df['avg_pm10']).astype('int')

# Merge, using both 'county' and 'category' as keys
df = pd.merge(df, averages, on=['county', 'category'], suffixes=('', '_avg'))

In [275]:
df.loc[df['sensor_id'] == 77]

,sensor_id,latitude,longitude,altitude,avg_pm2,avg_pm10,category,county,cat_avg_pm2,cat_avg_pm10
0,77,40.750816,-111.82529,NaN,18,18,red,Salt Lake County,669,670


In [276]:
selected_colors =['red','orange','green','yellow']
df.loc[df['category'].isin(selected_colors)]

,sensor_id,latitude,longitude,altitude,avg_pm2,avg_pm10,category,county,cat_avg_pm2,cat_avg_pm10
0,77,40.750816,-111.825290,NaN,18,18,red,Salt Lake County,669,670
1,443,41.224422,-111.968376,NaN,7,7,red,Weber County,5,6
2,525,40.783870,-111.870140,4891.0,1,1,green,Salt Lake County,51,51
5,3388,40.733440,-111.821510,4715.0,0,0,green,Salt Lake County,51,51
7,5174,41.172134,-111.923890,5188.0,2,2,green,Weber County,240,240
...,...,...,...,...,...,...,...,...,...,...
190,156037,40.722088,-111.818810,4698.0,2,3,orange,Salt Lake County,166,166
191,156623,40.746613,-111.847490,4598.0,1,1,green,Salt Lake County,51,51
195,169779,40.769947,-111.847015,4758.0,2,3,red,Salt Lake County,669,670
196,169851,40.902760,-111.872420,4361.0,2,3,orange,Davis County,6,6


In [240]:
df_color.category.unique()

array(['red', 'yellow', 'green', 'orange'], dtype=object)

In [56]:
# Convert the date column to datetime
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')

# Now you can sort by this column
df_sorted = df.sort_values(by='date')

In [60]:
df_sorted.groupby('date').count().reset_index()[['date']].to_csv('date_range.csv', index=False)

In [319]:
def return_county(begin_date, end_date,red,orange,green,lightBlue):
    # Set up sqlite
    connection = sqlite3.connect('../static/data/sensors_readings_2016_present.db')
    
    # Assemble Query with proper placement of WHERE clause and using parameterized queries
    sql_query = """
    SELECT sensor_id, latitude, longitude, altitude, AVG(pm2) AS avg_pm2, AVG(pm10) AS avg_pm10
    FROM sensors_readings
    WHERE (date BETWEEN ? AND ?)
    GROUP BY sensor_id, latitude, longitude, altitude
    """
    
    # Execute the query with parameter substitution to prevent SQL injection
    df = pd.read_sql_query(sql_query, connection, params=(begin_date, end_date))
    
    connection.close()
    
    # Join with color categories
    df_color = pd.read_csv('../static/data/sensor_categories.csv')
    df = pd.merge(df,df_color, on = 'sensor_id')
    
    selected_colors =[]
    if red == True:
        selected_colors.append('red')
    if orange == True:
        selected_colors.append('orange')
    if green == True:
        selected_colors.append('green')
    if lightBlue == True:
        selected_colors.append('blue')
    
    df = df.loc[df['category'].isin(selected_colors)]
    
    df.drop(['category'], axis=1, inplace=True)
    # Get category averages
    averages = df.groupby('county').mean().reset_index()
    averages = averages[['county','avg_pm2','avg_pm10']]
    averages.rename(columns = {'avg_pm2':'cat_avg_pm2','avg_pm10':'cat_avg_pm10'}, inplace = True)
    averages['cat_avg_pm2'] = round(averages['cat_avg_pm2']).astype('int')
    averages['cat_avg_pm10'] = round(averages['cat_avg_pm10']).astype('int')
    
    # Join
    
    df['avg_pm2'] = round(df['avg_pm2']).astype('int')
    df['avg_pm10'] = round(df['avg_pm10']).astype('int')

    df = pd.merge(df,averages, on = 'county')
    df = pd.merge(df,df_color, on = ['sensor_id','county'])
    
    return df

In [320]:
df = return_county('2023-09-27','2023-09-30',True,True,True,True)
df.cat_avg_pm2.unique()

array([131, 241, 141])

In [321]:
df

,sensor_id,latitude,longitude,altitude,avg_pm2,avg_pm10,county,cat_avg_pm2,cat_avg_pm10,category
0,77,40.750816,-111.825290,4870,18,18,Salt Lake County,131,132,red
1,443,41.224422,-111.968376,4565,7,7,Weber County,241,241,red
2,525,40.783870,-111.870140,4891,1,1,Salt Lake County,131,132,green
3,984,40.595387,-111.807755,5149,2,2,Salt Lake County,131,132,blue
4,3238,40.664246,-111.849230,4331,3,3,Salt Lake County,131,132,blue
...,...,...,...,...,...,...,...,...,...,...
178,165997,40.733875,-111.841090,4561,2,3,Salt Lake County,131,132,blue
179,168215,40.677616,-111.834690,4458,2,3,Salt Lake County,131,132,blue
180,169779,40.769947,-111.847015,4758,2,3,Salt Lake County,131,132,red
181,169851,40.902760,-111.872420,4361,2,3,Davis County,141,141,orange


In [300]:
2+2

4

In [314]:
df_color

,sensor_id,category,county
0,77,red,Salt Lake County
1,443,red,Weber County
2,525,green,Salt Lake County
3,984,blue,Salt Lake County
4,992,green,Salt Lake County
...,...,...,...
217,204009,red,Salt Lake County
218,204315,green,Salt Lake County
219,207743,blue,Salt Lake County
220,208781,green,Weber County


In [322]:
def return_table(begin_date, end_date,red,orange,green,lightBlue):
    # Set up sqlite
    connection = sqlite3.connect('../static/data/sensors_readings_2016_present.db')
    
    # Assemble Query
    sql_query = """
    SELECT sensor_id, latitude, longitude, altitude, AVG(pm2) AS avg_pm2, AVG(pm10) AS avg_pm10
    FROM sensors_readings
    WHERE date(date) BETWEEN ? AND ?
    GROUP BY sensor_id, latitude, longitude, altitude
    """
    
    # Execute the query
    df = pd.read_sql_query(sql_query, connection, params=(begin_date, end_date))
    
    # Join with color categories
    df_color = pd.read_csv('../static/data/sensor_categories.csv')
    df = pd.merge(df,df_color, on = 'sensor_id')
    df.drop(['county'], axis=1, inplace=True)
    # Get category averages
    averages = df.groupby('category').mean().reset_index()
    averages = averages[['category','avg_pm2','avg_pm10']]
    averages.rename(columns = {'avg_pm2':'cat_avg_pm2','avg_pm10':'cat_avg_pm10'}, inplace = True)
    averages['cat_avg_pm2'] = round(averages['cat_avg_pm2']).astype('int')
    averages['cat_avg_pm10'] = round(averages['cat_avg_pm10']).astype('int')
    
    # Join
    
    df['avg_pm2'] = round(df['avg_pm2']).astype('int')
    df['avg_pm10'] = round(df['avg_pm10']).astype('int')

    df = pd.merge(df,averages, on = 'category')
    
    
    
    selected_colors =[]
    if red == True:
        selected_colors.append('red')
    if orange == True:
        selected_colors.append('orange')
    if green == True:
        selected_colors.append('green')
    if lightBlue == True:
        selected_colors.append('blue')
    
    df_income_colors = df.loc[df['category'].isin(selected_colors)]
    df = pd.merge(df,df_color, on = ['sensor_id','category'])
    
    connection.close()
    return df_income_colors
    

In [323]:
df = return_county('2023-09-27','2023-09-30',True,True,True,True)
df

,sensor_id,latitude,longitude,altitude,avg_pm2,avg_pm10,county,cat_avg_pm2,cat_avg_pm10,category
0,77,40.750816,-111.825290,4870,18,18,Salt Lake County,131,132,red
1,443,41.224422,-111.968376,4565,7,7,Weber County,241,241,red
2,525,40.783870,-111.870140,4891,1,1,Salt Lake County,131,132,green
3,984,40.595387,-111.807755,5149,2,2,Salt Lake County,131,132,blue
4,3238,40.664246,-111.849230,4331,3,3,Salt Lake County,131,132,blue
...,...,...,...,...,...,...,...,...,...,...
178,165997,40.733875,-111.841090,4561,2,3,Salt Lake County,131,132,blue
179,168215,40.677616,-111.834690,4458,2,3,Salt Lake County,131,132,blue
180,169779,40.769947,-111.847015,4758,2,3,Salt Lake County,131,132,red
181,169851,40.902760,-111.872420,4361,2,3,Davis County,141,141,orange
